# Get Places

We will study the layout of the output from the API

In [1]:
import pandas as pd
import json
import folium
import itertools
import os

In [2]:
# Load json 
walk_path: str = "../tmp/"
samples: list = list()
for _,_, files in os.walk(walk_path):
    for file in files:
        if not os.path.isdir(file):
            fullpath = os.path.join(walk_path, file)
            with open(fullpath) as f:
                loaded_var: dict = json.load(f)
            results: list[dict] = loaded_var['results']
            samples.append(results)
        else: 
            pass

unpacked_samples = list(itertools.chain(*samples))

# So now we have our requests listed inside a list variable
We gotta load it into a DataFrame, define a basic structure and upload it to BigQuery

In [3]:
df = pd.DataFrame(unpacked_samples)
df.head(1)

,fsq_id,categories,features,geocodes,hours,hours_popular,location,name,rating,social_media,venue_reality_bucket,website,price,description
0,4c192f2e4ff90f473a8c0f49,"[{'id': 10051, 'name': 'Stadium', 'short_name'...",{'payment': {'credit_cards': {'accepts_credit_...,"{'main': {'latitude': 51.184392, 'longitude': ...","{'display': 'Open Daily 0:00-2:00, 8:00-24:00'...","[{'close': '2100', 'day': 1, 'open': '1600'}, ...","{'address': 'Julius De Geyterstraat 133', 'cou...",T.C. Olympia Beerschot,7.5,{},VeryHigh,http://www.tennis7deolympiade.be,NaN,NaN


In [4]:
# Extracting only the info we want from the category
df.loc[:, "categories"] = df['categories'].apply(lambda cats: cats[0]['short_name'] if cats and 'short_name' in cats[0] else None)
df.drop(columns = ["categories"], inplace = True)

For the geocodes

In [5]:
df.geocodes[0]

{'main': {'latitude': 51.184392, 'longitude': 4.380101},
 'roof': {'latitude': 51.184392, 'longitude': 4.380101}}

In [6]:
df.loc[:, "latitude"] = df.geocodes.apply(lambda dic: dic['main']['latitude'])
df.loc[:, "longitude"] = df.geocodes.apply(lambda dic: dic['main']['longitude'])
df.drop(columns = ["geocodes"], inplace = True)

For locations object. Here we can extract some interesting information

In [7]:
df.location[0]

{'address': 'Julius De Geyterstraat 133',
 'country': 'BE',
 'cross_street': '7de Olympiadelaan',
 'formatted_address': 'Julius De Geyterstraat 133 (7de Olympiadelaan), 2020 Antwerpen',
 'locality': 'Wilrijk',
 'postcode': '2020',
 'region': 'Provincie Antwerpen'}

In [8]:
# Extracting all fields but formatted address (concatenated data only)
df.loc[:, "address"] = df.location.apply(lambda dic: dic.get("address"))
df.loc[:, "country"] = df.location.apply(lambda dic: dic.get('country'))
df.loc[:, "locality"] = df.location.apply(lambda dic: dic.get('locality'))
df.loc[:, "postcode"] = df.location.apply(lambda dic: dic.get('postcode'))
df.loc[:, "region"] = df.location.apply(lambda dic: dic.get('region'))
df.drop(columns = ["location"], inplace = True)

In [9]:
df.head(5)

,fsq_id,features,hours,hours_popular,name,rating,social_media,venue_reality_bucket,website,price,description,latitude,longitude,address,country,locality,postcode,region
0,4c192f2e4ff90f473a8c0f49,{'payment': {'credit_cards': {'accepts_credit_...,"{'display': 'Open Daily 0:00-2:00, 8:00-24:00'...","[{'close': '2100', 'day': 1, 'open': '1600'}, ...",T.C. Olympia Beerschot,7.5,{},VeryHigh,http://www.tennis7deolympiade.be,NaN,NaN,51.184392,4.380101,Julius De Geyterstraat 133,BE,Wilrijk,2020,Provincie Antwerpen
1,4e1be4d16284102ec19833df,{},"{'is_local_holiday': False, 'open_now': False}","[{'close': '1200', 'day': 1, 'open': '1100'}, ...",Jamil,7.4,{},VeryHigh,http://www.prima.be,NaN,NaN,51.183157,4.379273,None,BE,Antwerpen,None,Provincie Antwerpen
2,5b9d01ff772fbc002cf25023,{},"{'display': 'Tue-Sun 8:30-22:00', 'is_local_ho...",NaN,Okka,8.0,{},High,http://okka.be,2.0,NaN,51.184223,4.373664,Sint-Bernardsesteenweg 313,BE,Provincie Antwerpen,2660,Provincie Antwerpen
3,4cc83111afcfb713445b7089,{'payment': {'credit_cards': {'accepts_credit_...,"{'display': 'Thu 17:00-19:00', 'is_local_holid...","[{'close': '2100', 'day': 3, 'open': '1900'}, ...",Olympisch Stadion,7.7,"{'facebook_id': '116625381718227', 'twitter': ...",VeryHigh,http://www.beerschotwilrijk.be,NaN,NaN,51.183998,4.382336,Atletenstraat 80,BE,Antwerpen,2020,Provincie Antwerpen
4,4cdadc8899026dcb8b0a0981,{'payment': {'credit_cards': {'accepts_credit_...,"{'is_local_holiday': False, 'open_now': False}","[{'close': '2200', 'day': 1, 'open': '1700'}, ...",Ling Nan bvba,8.2,{'facebook_id': '372364609448661'},VeryHigh,http://www.lingnan.be,1.0,NaN,51.183249,4.382782,VIIde Olympiadelaan 138,BE,Kiel,2020,Provincie Antwerpen


In [10]:
df.features[0]
# This field has only information about credit card usage. We will just extract data from it when it does

{'payment': {'credit_cards': {'accepts_credit_cards': True}}}

In [11]:
# Features treatment
def unpack_credit_card(obj: dict or float) -> bool or float:
    if isinstance(obj, float):
        return None
    else:
        try:
            var: bool = bool(obj.get("payment").get("credit_cards").get("accepts_credit_cards"))
        except Exception as e:
            var: float = None
        return var
        
df.loc[:, "accepts_pinpas"] = df.features.apply(lambda obj: unpack_credit_card(obj))
df.drop(columns = ["features"], inplace = True)

In [12]:
df.head(1)

,fsq_id,hours,hours_popular,name,rating,social_media,venue_reality_bucket,website,price,description,latitude,longitude,address,country,locality,postcode,region,accepts_pinpas
0,4c192f2e4ff90f473a8c0f49,"{'display': 'Open Daily 0:00-2:00, 8:00-24:00'...","[{'close': '2100', 'day': 1, 'open': '1600'}, ...",T.C. Olympia Beerschot,7.5,{},VeryHigh,http://www.tennis7deolympiade.be,NaN,NaN,51.184392,4.380101,Julius De Geyterstraat 133,BE,Wilrijk,2020,Provincie Antwerpen,True


### Social media treatment

In [13]:
[item.keys() for item in df.social_media][4:15]

[dict_keys(['facebook_id']),
 dict_keys(['facebook_id', 'instagram', 'twitter']),
 dict_keys([]),
 dict_keys(['facebook_id']),
 dict_keys([]),
 dict_keys([]),
 dict_keys(['facebook_id']),
 dict_keys(['facebook_id']),
 dict_keys(['facebook_id', 'instagram', 'twitter']),
 dict_keys([]),
 dict_keys([])]

In [14]:
def unpack_social_media(obj:dict or float, media: str) -> bool:
    medias_obj = {
        "facebook" : "facebook_id",
        "instagram" :  "instagram",
        "twitter" : "twitter"
    }
    assert media in medias_obj.keys(), "Wrong media."
    
    if isinstance(obj, float) or obj == {}:
        return None
    else:
        key: str = medias_obj[media]
        return obj.get(key)

df.loc[:, "facebook_id"] = df.social_media.apply(lambda obj: unpack_social_media(obj, "facebook"))
df.loc[:, "instagram"] = df.social_media.apply(lambda obj: unpack_social_media(obj, "instagram"))      
df.loc[:, "twitter"] = df.social_media.apply(lambda obj: unpack_social_media(obj, "twitter"))
df.drop(columns = ['social_media'], inplace = True)

In [15]:
df.hours[3]

{'display': 'Thu 17:00-19:00',
 'is_local_holiday': False,
 'open_now': False,
 'regular': [{'close': '1900', 'day': 4, 'open': '1700'}]}

#### Regular Hours Treatment

In [16]:
df.loc[:, "hours_display"] = df.hours.apply(lambda dic: dic.get('display'))
df.loc[:, "hours_regular"] = df.hours.apply(lambda dic: dic.get('regular'))
df.drop(columns = ["hours"], inplace = True)

In [17]:
df.hours_popular[0]

[{'close': '2100', 'day': 1, 'open': '1600'},
 {'close': '2200', 'day': 2, 'open': '1600'},
 {'close': '2100', 'day': 3, 'open': '1600'},
 {'close': '2300', 'day': 4, 'open': '1500'},
 {'close': '2200', 'day': 5, 'open': '1600'},
 {'close': '2100', 'day': 6, 'open': '0900'},
 {'close': '2000', 'day': 7, 'open': '1100'}]

In [18]:
df.dtypes

fsq_id                   object
hours_popular            object
name                     object
rating                  float64
venue_reality_bucket     object
website                  object
price                   float64
description              object
latitude                float64
longitude               float64
address                  object
country                  object
locality                 object
postcode                 object
region                   object
accepts_pinpas           object
facebook_id              object
instagram                object
twitter                  object
hours_display            object
hours_regular            object
dtype: object

Now we got to ensure all columns have a single type. That will make the export easier to bigquery

In [19]:
df.columns

Index(['fsq_id', 'hours_popular', 'name', 'rating', 'venue_reality_bucket',
       'website', 'price', 'description', 'latitude', 'longitude', 'address',
       'country', 'locality', 'postcode', 'region', 'accepts_pinpas',
       'facebook_id', 'instagram', 'twitter', 'hours_display',
       'hours_regular'],
      dtype='object')

In [20]:
def convert_columns(df):
    dtype_mapping = {
        'fsq_id': str,
        'name': str,
        'venue_reality_bucket': str,
        'website': str,
        'price': str,
        'description': str,
        'address': str,
        'country': str,
        'locality': str,
        'region': str,
        'facebook_id': str,
        'instagram': str,
        'twitter': str,
        'hours_display': str,
        'rating': float,
        'latitude': float,
        'longitude': float,
        'postcode': 'Int64',  # Nullable integer type
        'accepts_pinpas': bool,
        'hours_popular': object,  # Will keep as list[dict]
        'hours_regular': object   # Will keep as list[dict]
    }
    
    # Apply type conversions
    for col, dtype in dtype_mapping.items():
        if col in df.columns:
            if dtype == str:
                df[col] = df[col].astype(str)
            elif dtype in [float, int]:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            elif dtype == 'Int64':
                df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
    
    return df

df = convert_columns(df)

In [21]:
df['hours_popular'] = df['hours_popular'].apply(json.dumps)
df['hours_regular'] = df['hours_regular'].apply(json.dumps)

In [25]:
df.loc[0]

fsq_id                                           4c192f2e4ff90f473a8c0f49
hours_popular           [{"close": "2100", "day": 1, "open": "1600"}, ...
name                                               T.C. Olympia Beerschot
rating                                                                7.5
venue_reality_bucket                                             VeryHigh
website                                  http://www.tennis7deolympiade.be
price                                                                 nan
description                                                           nan
latitude                                                        51.184392
longitude                                                        4.380101
address                                        Julius De Geyterstraat 133
country                                                                BE
locality                                                          Wilrijk
postcode                              